In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import f1_score, accuracy_score

In [2]:
test = pd.read_csv('../input/digit-recognizer/test.csv')
train = pd.read_csv('../input/digit-recognizer/train.csv')

In [3]:
y_train = train['label']
X_train = train.drop(labels = ['label'], axis=1)

In [4]:
train.head()

# EDA

In [5]:
train.isna().any().any()

In [6]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.countplot(data=train, x='label', palette='RdYlBu')

In [7]:
img = X_train.iloc[42].to_numpy()
img = img.reshape((28,28))
plt.axis('off')
plt.title('The answer to life, the universe, and everything')
plt.imshow(img,cmap='gray')

In [8]:
img = X_train.iloc[41999].to_numpy()
img = img.reshape((28,28))
plt.axis('off')
plt.title('The final digit')
plt.imshow(img,cmap=plt.cm.binary)

# Preparing data

In [9]:
X_train = X_train / 255
test = test / 255
print('Shape of training data:    ', X_train.shape)
print('Shape of test data:        ', test.shape)

X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
print('New shape of training data:', X_train.shape)
print('New shape of test data:    ', test.shape)

In [10]:
y_train = to_categorical(y_train, num_classes = 10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=2)
print("x_train shape: ",X_train.shape)
print("x_test shape:  ",X_val.shape)
print("y_train shape: ",y_train.shape)
print("y_test shape:  ",y_val.shape)

# ConvNet

In [11]:
model = Sequential()
model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size = 4, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [12]:
model.summary()

In [13]:
batch_size = 32
epochs = 100

In [14]:
datagen = ImageDataGenerator(
          rotation_range=10,
          zoom_range = 0.1,
          width_shift_range=0.1,
          height_shift_range=0.1)

datagen.fit(X_train)

In [ ]:
history = model.fit(datagen.flow(X_train,y_train, batch_size=batch_size),
                    epochs = epochs, 
                    validation_data = (X_val,y_val), 
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    verbose=1,
                    )

In [ ]:
history = pd.DataFrame(history.history)
print('Model Performance:')
print(('Objective: Accuracy: {:0.4f}')\
     .format(history['accuracy'].max()))
print(('Validation Accuracy: {:0.4f}')\
     .format(history['val_accuracy'].max()))
print(('Loss               : {:0.4f}')\
     .format(history['loss'].min()))
print(('Validation Loss    : {:0.4f}')\
     .format(history['val_loss'].min()))